<a href="https://colab.research.google.com/github/PPM5658/Book-recommender/blob/main/vector_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma
import pandas as pd
import numpy as np


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Books recommendation system/books_cleaned.csv')

In [ ]:
df['tagged_description'].to_csv("text_description.txt",sep="\n",index=False,header=False)

In [ ]:
raw_document=TextLoader('/content/text_description.txt').load()
text_spliter=CharacterTextSplitter(chunk_size=0.1,chunk_overlap=0,separator="\n")
documents=text_spliter.split_documents(raw_document)

In [ ]:
int(documents[0].page_content.split()[0].strip())

9780002005883

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'}  # Use 'cuda' for GPU
)
db_books=Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)

In [ ]:
def retrieve_books(query,k:int=10)->pd.DataFrame:
  docs=db_books.similarity_search(query,k=k)
  book_list=[]
  for i in range(0,len(docs)):
      book_list+=[int(docs[i].page_content.strip('"').split()[0])]
  return df[df['isbn13'].isin(book_list)].head(k)

In [ ]:
query="A book to teach children about nature"
retrieve_books(query,k=10)

In [ ]:
category_mapping = {'Fiction' : "Fiction",
 'Juvenile Fiction': "Children's Fiction",
 'Biography & Autobiography': "Nonfiction",
 'History': "Nonfiction",
 'Literary Criticism': "Nonfiction",
 'Philosophy': "Nonfiction",
 'Religion': "Nonfiction",
 'Comics & Graphic Novels': "Fiction",
 'Drama': "Fiction",
 'Juvenile Nonfiction': "Children's Nonfiction",
 'Science': "Nonfiction",
 'Poetry': "Fiction"}

df["simple_categories"] = df["categories"].map(category_mapping)

In [ ]:
df[~df['simple_categories'].isna()]

In [ ]:
from transformers import pipeline
fiction_categories=["Fiction","Nonfiction"]
pipe=pipeline("zero-shot-classification",model="facebook/bart-large-mnli",device=0)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
sequence=df.loc[df['simple_categories']=='Fiction','description'].reset_index(drop=True)[0]

In [ ]:
def generate_prediction(sequence,categories):
  prediction=pipe(sequence,fiction_categories)
  max_index=np.argmax(prediction['scores'])
  max_label=pipe(sequence,fiction_categories)['labels'][max_index]
  return max_label

In [ ]:
from tqdm import tqdm

In [ ]:
actual_cats = []
predicted_cats = []

# Populate actual_cats and predicted_cats for 'Fiction' category
for i in tqdm(range(300)):
  sequence = df.loc[df['simple_categories'] == 'Fiction', 'description'].reset_index(drop=True)[i]
  predicted_cats.append(generate_prediction(sequence, fiction_categories))
  actual_cats.append('Fiction')

# Populate actual_cats and predicted_cats for 'Nonfiction' category
for i in tqdm(range(300)):
  sequence = df.loc[df['simple_categories'] == 'Nonfiction', 'description'].reset_index(drop=True)[i]
  predicted_cats.append(generate_prediction(sequence, fiction_categories))
  actual_cats.append('Nonfiction')

# Create the DataFrame from the correctly populated lists
predicted = pd.DataFrame({'predicted_cats': predicted_cats, 'actual_cats': actual_cats})

# Calculate accuracy
predicted['correction'] = np.where(predicted['actual_cats'] == predicted['predicted_cats'], 1, 0)
accuracy = predicted['correction'].sum() / len(predicted)
print(f"Accuracy: {accuracy:.2f}")

100%|██████████| 300/300 [00:52<00:00,  5.70it/s]

Accuracy: 0.78


In [ ]:
isbn=[]
predictions=[]
missing_cats=df.loc[df['simple_categories'].isna(),['isbn13','description']].reset_index(drop=True)

In [ ]:
for i in tqdm(range(0, len(missing_cats))):
    sequence = missing_cats["description"][i]
    predictions += [generate_prediction(sequence, fiction_categories)]
    isbn += [missing_cats["isbn13"][i]]

100%|██████████| 1454/1454 [04:07<00:00,  5.87it/s]


In [ ]:
missing_predicted_df=pd.DataFrame({'isbn13':isbn,'predicted_cats':predictions})

In [ ]:
len(predictions)
len(isbn)

1454

In [ ]:
books=pd.merge(df,missing_predicted_df,how='left',on='isbn13')
books['simple_categories']=np.where(books['simple_categories'].isna(),books['predicted_cats'],books['simple_categories'])
books=books.drop(columns=['predicted_cats'])

In [ ]:
books

In [ ]:
books[books["categories"].str.lower().isin([
    "romance",
    "science fiction",
    "scifi",
    "fantasy",
    "horror",
    "mystery",
    "thriller",
    "comedy",
    "crime",
    "historical"
])]

In [ ]:
books.to_csv("books_with_categories.csv", index=False)

In [ ]:
books = pd.read_csv("books_with_categories.csv")
from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device = 0)
classifier("I love this!")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.00852868054062128},
  {'label': 'neutral', 'score': 0.005764591973274946},
  {'label': 'anger', 'score': 0.004419785924255848},
  {'label': 'sadness', 'score': 0.0020923891570419073},
  {'label': 'disgust', 'score': 0.001611991785466671},
  {'label': 'fear', 'score': 0.00041385178337804973}]]

In [ ]:
classifier([books['description'][0]])

[[{'label': 'fear', 'score': 0.6548413634300232},
  {'label': 'neutral', 'score': 0.16985207796096802},
  {'label': 'sadness', 'score': 0.11640888452529907},
  {'label': 'surprise', 'score': 0.02070062793791294},
  {'label': 'disgust', 'score': 0.019100705161690712},
  {'label': 'joy', 'score': 0.015161297284066677},
  {'label': 'anger', 'score': 0.003935146611183882}]]

In [ ]:
classifier(books['description'][0].split('.'))

[[{'label': 'surprise', 'score': 0.7296026945114136},
  {'label': 'neutral', 'score': 0.1403856873512268},
  {'label': 'fear', 'score': 0.06816219538450241},
  {'label': 'joy', 'score': 0.04794241115450859},
  {'label': 'anger', 'score': 0.009156348183751106},
  {'label': 'disgust', 'score': 0.00262847519479692},
  {'label': 'sadness', 'score': 0.0021221605129539967}],
 [{'label': 'neutral', 'score': 0.44937071204185486},
  {'label': 'disgust', 'score': 0.2735914885997772},
  {'label': 'joy', 'score': 0.10908304899930954},
  {'label': 'sadness', 'score': 0.09362724423408508},
  {'label': 'anger', 'score': 0.040478333830833435},
  {'label': 'surprise', 'score': 0.02697017975151539},
  {'label': 'fear', 'score': 0.006879060063511133}],
 [{'label': 'neutral', 'score': 0.6462162137031555},
  {'label': 'sadness', 'score': 0.2427332103252411},
  {'label': 'disgust', 'score': 0.04342261329293251},
  {'label': 'surprise', 'score': 0.028300540521740913},
  {'label': 'joy', 'score': 0.0142114423

In [ ]:
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores={label: [] for label in emotion_labels}
def calculate_max_emotions_score(predictions):
  per_emotion_scores={label: [] for label in emotion_labels}
  for prediction in predictions:
    sorted_predictions=sorted(prediction,key=lambda x:x["label"])
    for index,label in enumerate(emotion_labels):
      per_emotion_scores[label].append(sorted_predictions[index]["score"])
  return {label:np.max(score) for label,score in per_emotion_scores.items()}

In [ ]:
for i in range(10):
  isbn.append(books['isbn13'][i])
  sentences=books['description'][i].split('.')
  prediction=classifier(sentences)
  max_scores=calculate_max_emotions_score(prediction)
  for label in emotion_labels:
    emotion_scores[label].append(max_scores[label])

In [ ]:
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores={label: [] for label in emotion_labels}
for i in tqdm(range(len(books))):
  isbn.append(books['isbn13'][i])
  sentences=books['description'][i].split('.')
  prediction=classifier(sentences)
  max_scores=calculate_max_emotions_score(prediction)
  for label in emotion_labels:
    emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [02:16<00:00, 38.02it/s]


In [ ]:
emotions_df=pd.DataFrame(emotion_scores)
emotions_df['isbn13']=isbn

In [ ]:
emotions_df.head()

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273591,0.928168,0.932797,0.646216,0.967158,0.729603,9780002005883
1,0.612619,0.348284,0.942528,0.704422,0.887939,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078766,9780006178736
3,0.351484,0.150723,0.360706,0.251881,0.732685,0.111690,0.078766,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078766,9780006280934


In [ ]:
books=pd.merge(books,emotions_df,on='isbn13')
books

In [ ]:
books.to_csv("books_with_emotions.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma
import gradio as gr

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'}  # Use 'cuda' for GPU
)

load_dotenv()

books = pd.read_csv("books_with_emotions.csv")
books["large_thumbnail"] = books["thumbnail"] + "&fife=w800"
books["large_thumbnail"] = np.where(
    books["large_thumbnail"].isna(),
    "cover-not-found.jpg",
    books["large_thumbnail"],
)

raw_documents = TextLoader("/content/text_description.txt").load()
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=0.1, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db_books = Chroma.from_documents(documents, embeddings)


def retrieve_semantic_recommendations(
        query: str,
        category: str = None,
        tone: str = None,
        initial_top_k: int = 50,
        final_top_k: int = 16,
) -> pd.DataFrame:

    recs = db_books.similarity_search(query, k=initial_top_k)
    books_list = [int(rec.page_content.strip('"').split()[0]) for rec in recs]
    book_recs = books[books["isbn13"].isin(books_list)].head(initial_top_k)

    if category != "All":
        book_recs = book_recs[book_recs["simple_categories"] == category].head(final_top_k)
    else:
        book_recs = book_recs.head(final_top_k)

    if tone == "Happy":
        book_recs.sort_values(by="joy", ascending=False, inplace=True)
    elif tone == "Surprising":
        book_recs.sort_values(by="surprise", ascending=False, inplace=True)
    elif tone == "Angry":
        book_recs.sort_values(by="anger", ascending=False, inplace=True)
    elif tone == "Suspenseful":
        book_recs.sort_values(by="fear", ascending=False, inplace=True)
    elif tone == "Sad":
        book_recs.sort_values(by="sadness", ascending=False, inplace=True)

    return book_recs


def recommend_books(
        query: str,
        category: str,
        tone: str
):
    recommendations = retrieve_semantic_recommendations(query, category, tone)
    results = []

    for _, row in recommendations.iterrows():
        description = row["description"]
        truncated_desc_split = description.split()
        truncated_description = " ".join(truncated_desc_split[:30]) + "..."

        authors_split = row["authors"].split(";")
        if len(authors_split) == 2:
            authors_str = f"{authors_split[0]} and {authors_split[1]}"
        elif len(authors_split) > 2:
            authors_str = f"{', '.join(authors_split[:-1])}, and {authors_split[-1]}"
        else:
            authors_str = row["authors"]

        caption = f"{row['title']} by {authors_str}: {truncated_description}"
        results.append((row["large_thumbnail"], caption))
    return results

categories = ["All"] + sorted(books["simple_categories"].unique())
tones = ["All"] + ["Happy", "Surprising", "Angry", "Suspenseful", "Sad"]

with gr.Blocks(theme = gr.themes.Glass()) as dashboard:
    gr.Markdown("# Semantic book recommender")

    with gr.Row():
        user_query = gr.Textbox(label = "Please enter a description of a book:",
                                placeholder = "e.g., A story about forgiveness")
        category_dropdown = gr.Dropdown(choices = categories, label = "Select a category:", value = "All")
        tone_dropdown = gr.Dropdown(choices = tones, label = "Select an emotional tone:", value = "All")
        submit_button = gr.Button("Find recommendations")

    gr.Markdown("## Recommendations")
    output = gr.Gallery(label = "Recommended books", columns = 8, rows = 2)

    submit_button.click(fn = recommend_books,
                        inputs = [user_query, category_dropdown, tone_dropdown],
                        outputs = output)


if __name__ == "__main__":
    dashboard.launch()

Streaming output truncated to the last 5000 lines.


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0582c0aec8c1c58d9c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
